<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
# ! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 
! pip install -q deepmatch

--2022-09-21 19:23:12--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  13.2MB/s    in 0.4s    

2022-09-21 19:23:12 (13.2 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-21 19:23:12--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [2]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

# Data 

In [3]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [4]:
data.user_id.nunique(), data.movie_id.nunique(), data.head(2)

(46, 2175,    user_id  movie_id  rating  timestamp  \
 0        1      1193       5  978300760   
 1        1       661       3  978302109   
 
                                     title     genres gender  age  occupation  \
 0  One Flew Over the Cuckoo's Nest (1975)      Drama      F    1          10   
 1        James and the Giant Peach (1996)  Animation      F    1          10   
 
      zip  
 0  48067  
 1  48067  )

# Feature Engineering

In [5]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
negsample = 1


In [6]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
feature_max_idx 

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [7]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18
2,1,483,3,978301968,My Fair Lady (1964),12,1,1,9,18
3,1,1881,4,978300275,Erin Brockovich (2000),8,1,1,9,18
4,1,1306,5,978824291,"Bug's Life, A (1998)",3,1,1,9,18


In [8]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')

In [9]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [10]:
train_set, test_set = gen_data_set(data, SEQ_LEN, negsample)
len(train_set), len(test_set)

100%|██████████| 46/46 [00:00<00:00, 111.68it/s]

7 8


(19816, 46)

In [11]:
data.loc[2]

user_id                         1
movie_id                      483
rating                          3
timestamp               978301968
title         My Fair Lady (1964)
genres                         12
gender                          1
age                             1
occupation                      9
zip                            18
Name: 2, dtype: object

In [12]:
print(train_set[0])

# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - constant
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len,  - 50
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating

(4, 926, 0, [49, 1187, 1547, 912, 1833, 955, 1260, 1798, 988, 222, 843, 831, 662, 897, 873, 1367, 95, 898, 808, 357, 644, 2094, 2018, 1423, 1258, 1028, 156, 430, 1435, 1259, 990, 925, 265, 133, 780, 1716, 423, 1373, 338, 205, 1737, 958, 445, 599, 852, 838, 352, 1752, 1135, 1812], 50, [1, 1, 6, 6, 5, 6, 1, 5, 6, 5, 5, 6, 6, 6, 1, 1, 8, 8, 8, 8, 8, 5, 5, 5, 8, 13, 8, 5, 5, 1, 8, 8, 1, 5, 1, 11, 1, 8, 18, 5, 5, 5, 8, 13, 1, 5, 5, 8, 1, 5], 8)


In [13]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((19816,), (19816,), (46,), (46,))

In [14]:
train_model_input

{'user_id': array([ 4, 31, 18, ..., 10,  6, 29]),
 'movie_id': array([ 926, 2020,  701, ..., 1144, 1929,  529]),
 'hist_movie_id': array([[  49, 1187, 1547, ..., 1752, 1135, 1812],
        [1484,  457, 1577, ...,  644, 1189, 1858],
        [ 137,  478, 2028, ...,    0,    0,    0],
        ...,
        [1764,  929,  782, ...,  587, 1589, 1381],
        [ 719, 1617, 1957, ...,    2, 1600, 1082],
        [2084, 1591, 1984, ...,    0,    0,    0]], dtype=int32),
 'hist_genres': array([[ 1,  1,  6, ...,  8,  1,  5],
        [11,  1, 16, ...,  8,  8,  5],
        [ 5,  5,  5, ...,  0,  0,  0],
        ...,
        [ 2,  8,  1, ...,  5,  8, 11],
        [ 1,  1,  1, ...,  2,  1,  2],
        [ 3,  5,  8, ...,  0,  0,  0]], dtype=int32),
 'hist_len': array([50, 50, 31, ..., 50, 50, 35]),
 'genres': array([ 8,  1,  5, ...,  8,  5, 14]),
 'gender': array([2, 2, 2, ..., 2, 1, 2]),
 'age': array([3, 5, 6, ..., 5, 2, 4]),
 'occupation': array([ 6, 15,  2, ...,  4,  4, 16]),
 'zip': array([12, 28, 

# Aggregate Feature and create Embeddings

In [15]:
embedding_dim = 32

In [16]:
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

In [ ]:
VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
sparsefeat, maxlen, combiner="mean", length_name=None, weight_name=None, weight_norm=True):

In [17]:
item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]


In [18]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)


In [19]:
len(sampler_config.item_count)

2176

In [23]:
item_feature_columns[0].vocabulary_size

2176

# Model Training

In [24]:
import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

In [25]:
model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [31]:
model.user_input_list

AttributeError: ignored

In [22]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )


Train on 9908 samples
Epoch 1/20
9908/9908 [==============================] - 3s 302us/sample - loss: 10.9827
Epoch 2/20
9908/9908 [==============================] - 1s 131us/sample - loss: 8.8572
Epoch 3/20
9908/9908 [==============================] - 1s 132us/sample - loss: 8.1865
Epoch 4/20
9908/9908 [==============================] - 1s 125us/sample - loss: 8.1624
Epoch 5/20
9908/9908 [==============================] - 1s 145us/sample - loss: 7.5711
Epoch 6/20
9908/9908 [==============================] - 1s 137us/sample - loss: 7.1665
Epoch 7/20
9908/9908 [==============================] - 1s 115us/sample - loss: 6.9846
Epoch 8/20
9908/9908 [==============================] - 1s 115us/sample - loss: 6.6236
Epoch 9/20
9908/9908 [==============================] - 1s 103us/sample - loss: 6.3195
Epoch 10/20
9908/9908 [==============================] - 1s 61us/sample - loss: 6.1179
Epoch 11/20
9908/9908 [==============================] - 1s 62us/sample - loss: 5.9283
Epoch 12/20
9908/990

In [23]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(46, 32)
(2175, 32)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [24]:
user_embs

array([[ 0.0169137 , -0.10816157, -0.07978916, ...,  0.18221377,
        -0.18655513,  0.21925032],
       [-0.19826971, -0.1532387 ,  0.12011836, ...,  0.06905224,
        -0.18064997,  0.07048938],
       [-0.05367624, -0.07763619, -0.16798156, ...,  0.28834173,
         0.01302002,  0.1018568 ],
       ...,
       [-0.20442586, -0.0350772 ,  0.11478215, ...,  0.22919205,
         0.10099558, -0.09327433],
       [-0.090487  ,  0.11693648,  0.16743033, ...,  0.14837542,
         0.00444227,  0.19430703],
       [-0.1542987 , -0.21318945,  0.01800997, ...,  0.27099133,
        -0.05132304,  0.06733485]], dtype=float32)